In [6]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Define a function for pose detection using MediaPipe
def detect_pose(image):

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose_landmarks:
        results = pose_landmarks.process(image_rgb)
    
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    return image

def detect_pose_on_video():
    
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        frame_with_pose = detect_pose(frame.copy())

        cv2.imshow('Pose Detection', frame_with_pose)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def detect_pose_on_photo(photo_path):
    image = cv2.imread(photo_path)

    image_with_pose = detect_pose(image)

    cv2.imshow('Pose Detection', image_with_pose)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

mode = input("Enter 'video' to detect pose on video or 'photo' to detect pose on photo: ")

if mode == 'video':
    detect_pose_on_video()
elif mode == 'photo':
    photo_path = input("Enter the path to the photo: ")
    detect_pose_on_photo(photo_path)
else:
    print("Invalid mode. Please enter either 'video' or 'photo'.")


Enter 'video' to detect pose on video or 'photo' to detect pose on photo: video


In [5]:
import cv2
import requests
from PIL import Image
import numpy as np
import mediapipe as mp
from io import BytesIO

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def estimate_pose(frame):
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose_landmarks:
        results = pose_landmarks.process(frame_rgb)

    keypoints = []
    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            height, width, _ = frame.shape
            keypoint = (int(landmark.x * width), int(landmark.y * height))
            keypoints.append(keypoint)

    return keypoints

def download_image(url):
    response = requests.get(url)
    if response.status_code == 200:
        img = Image.open(BytesIO(response.content))
        return img
    else:
        print(f"Error downloading image from {url}")
        return None

def resize_and_pad(img, target_size):
    img = img.resize(target_size, Image.ANTIALIAS)
    return img

def overlay_image(background, overlay, keypoints, vêtement_size, clothing_type, is_photo=True):

    if len(keypoints) < 3:
#         print("Not enough keypoints detected. Showing cloth in the middle.")

        overlay_x = (background.width - overlay.width) // 2
        overlay_y = (background.height - overlay.height) // 2
    else:
        if is_photo:
            division_point = int(background.height * 0.58) if clothing_type == 'upper' else int(background.height * 0.90)
        else:
            division_point = int(background.height * 0.80) if clothing_type == 'upper' else int(background.height * 0.90)
        
        # Ignore the first 7 keypoints, which correspond to the face
        body_keypoints = keypoints[7:]

        if not body_keypoints:
#             print("No body keypoints detected. Showing cloth in the middle.")

            overlay_x = (background.width - overlay.width) // 2
            overlay_y = (background.height - overlay.height) // 2
        else:
            center_x = sum(keypoint[0] for keypoint in body_keypoints) // len(body_keypoints)
            center_y = sum(keypoint[1] for keypoint in body_keypoints) // len(body_keypoints)

            overlay_x = max(0, center_x - overlay.width // 2)
            overlay_y = max(0, center_y - overlay.height // 2)

            if clothing_type == 'upper':
                overlay_y = min(division_point - overlay.height, overlay_y)
            else:
                overlay_y = min(int((background.height - overlay.height)) + 6, int(overlay_y*1.4)+6)


    if overlay.mode != 'RGBA':
        overlay = overlay.convert('RGBA')

    background.paste(overlay, (overlay_x, overlay_y), overlay)



def capture_video(vêtement_img, clothing_type):

    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        keypoints = estimate_pose(frame)

        frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        overlay_image(frame_pil, vêtement_img, keypoints, vêtement_size, clothing_type, is_photo=False)
        frame_rgb = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)

        cv2.imshow('Virtual Try-On', frame_rgb)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


def try_on_photo(photo, vêtement_img, vêtement_size, clothing_type, is_photo=True):
    keypoints = estimate_pose(photo)

    frame_pil = Image.fromarray(cv2.cvtColor(photo, cv2.COLOR_BGR2RGB))
    overlay_image(frame_pil, vêtement_img, keypoints, vêtement_size, clothing_type, is_photo=True)
    frame_rgb = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)

    cv2.imshow('Virtual Try-On', frame_rgb)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


def choose_clothing_image():
    vêtement_url = input("Enter the URL of the clothing image: ")
    return vêtement_url

def resize_photo_to_screen(photo_path, screen_width, screen_height):
    frame = cv2.imread(photo_path)
    aspect_ratio = frame.shape[1] / frame.shape[0]
    if aspect_ratio > screen_width / screen_height:
        resized_frame = cv2.resize(frame, (screen_width, int(screen_width / aspect_ratio)))
    else:
        resized_frame = cv2.resize(frame, (int(screen_height * aspect_ratio), screen_height))

    return resized_frame


def resize_clothing_image(clothing_url, screen_width, screen_height):
    clothing_img = download_image(clothing_url)
    if clothing_img is None:
        return None
    
    clothing_width = int(screen_width / 3.0)
    clothing_height = int(screen_height / 2.5)

    resized_clothing_img = clothing_img.resize((clothing_width, clothing_height), Image.ANTIALIAS)

    return resized_clothing_img


# Ask user for input type (live video or photo)
input_type = input("Choose input type (live/photo): ").lower()

# Example usage:
screen_width = 800  # Set your screen width
screen_height = 780  # Set your screen height


if input_type == 'photo':
    # Ask user for photo path
    photo_path = input("Enter the path to the photo: ")
    resized_photo = resize_photo_to_screen(photo_path, screen_width, screen_height)

    # Ask user for upper or lower body clothing
    clothing_type = input("Choose clothing type (upper/lower): ").lower()
    if clothing_type not in ['upper', 'lower']:
        print("Invalid clothing type. Defaulting to upper body.")
        clothing_type = 'upper'

    # Ask user for clothing image URL
    vêtement_url = choose_clothing_image()
    resized_clothing_img = resize_clothing_image(vêtement_url, screen_width, screen_height)
    
    vêtement_size = (resized_clothing_img.width, resized_clothing_img.height)

    try_on_photo(resized_photo, resized_clothing_img, vêtement_size, clothing_type, is_photo=True)
    
elif input_type == 'live':
    # Ask user for upper or lower body clothing
    clothing_type = input("Choose clothing type (upper/lower): ").lower()
    if clothing_type not in ['upper', 'lower']:
        print("Invalid clothing type. Defaulting to upper body.")
        clothing_type = 'upper'

        
    # Ask user for clothing image URL
    vêtement_url = choose_clothing_image()
    resized_clothing_img = resize_clothing_image(vêtement_url, screen_width, screen_height)
    
    vêtement_size = (resized_clothing_img.width, resized_clothing_img.height)

    capture_video(resized_clothing_img, clothing_type)
    
else:
    print("Invalid input type.")

Choose input type (live/photo): live
Choose clothing type (upper/lower): upper
Enter the URL of the clothing image: https://github.com/Piyusharora2003/label/assets/98682478/30b83daa-786f-4e39-894d-78ed5a77efa6


C:\Users\TISHA\AppData\Local\Temp\ipykernel_37808\2968985208.py:138: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_clothing_img = clothing_img.resize((clothing_width, clothing_height), Image.ANTIALIAS)
